In [ ]:
from google.colab import auth
auth.authenticate_user()

**Installing Required Libraries**

In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0  70472      0 --:--:-- --:--:-- --:--:-- 72485
OK
29 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 29 not upgraded.
Need to get 12.1 MB of archives.
After this operation, 27.6 MB of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 123942 files and directories currently installed.)
Preparing to unpack .../gcsfuse_0.41.7_amd64.deb ...
Unpacking gcsfuse (0.41.7) ...
Setting up gcsfuse (0.41.7) ...


In [ ]:
!pip install laspy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 615 kB 24.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for laspy: filename=laspy-2.2.0-py3-none-any.whl size=63856 sha256=78a4eaabba3b00d58a45686570b734bc9cd6c738f7d7cd70e9b9c638d5658152
  Stored in directory: /root/.cache/pip/wheels/19/6e/01/0dfe379f345dfcfd77907d182cc2b6bae804b52eba66c028f6
Successfully built laspy


In [ ]:
!pip install pylas[lazrs,laszip]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 921 kB 37.6 MB/s 
  Created wheel for pylas: filename=pylas-0.4.3-py3-none-any.whl size=40353 sha256=29aa11c02c2bf1ee0d0a82d9e1ca21ce88461cd5f3f1a9272d615a7ead456128
  Stored in directory: /root/.cache/pip/wheels/b2/af/a6/c8c7ce1b3dc27174d5524f6c9cca5df59974caca07534d4408
Successfully built pylas


In [ ]:
!pip install fsspec

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install gcsfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 138 kB 27.0 MB/s 
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2022.8.2
    Uninstalling fsspec-2022.8.2:
      Successfully uninstalled fsspec-2022.8.2


**Import Libraries**

In [ ]:
import laspy
import numpy as np
import pandas as pd
from mpl_toolkits import mplot3d
import sys
import matplotlib.pyplot as plt
import datetime
import time
%matplotlib
import pylas

Using matplotlib backend: agg


**GPS to UTC Time Conversion Function**

In [ ]:
from datetime import datetime     #2010-10-26 23:58:34.728675 2010-10-26 23:58:34.728683  2010-10-27 00:41:34.539662
from datetime import timedelta
def gps_time_conversion(gps_time):
    x= gps_time + (10 ** 9)
    y=x/86400
    Begindatestring="1980-01-06"
    Begindate = datetime.strptime(Begindatestring, "%Y-%m-%d")
    Enddate = Begindate + timedelta(days=y)
    return Enddate.strftime("%Y-%m-%d %H:%M:%S")

**Connect and Download .LAZ files from GCP**

In [ ]:
from google.cloud import storage
import os

from google.colab import auth
auth.authenticate_user()
def download_file(filename):
  client = storage.Client()
  bucket = client.get_bucket('sanddiego2015')  
  blob = bucket.blob(filename)
  blob.download_to_filename(filename)
  print("i am the file",filename)

**To Convert .LAZ to LAS - Extract data points from LiDAR files**

In [ ]:
import os

import csv
  
# Open file 
import csv
with open('/content/Book3.csv', newline='') as csvfile:
  data = csv.reader(csvfile, delimiter=' ', quotechar='|')
  for row in data:
   a=', '.join(row)
   if('\ufeff' in a ):
     b=a.replace('\ufeff','')
     b=b.replace('./','')
   else:
    b=a.replace('./','')
   input_laz_path = '/content/'+ b 
   print()
   lat = []
   lon = []
   k=0
   download_file(b)

   laz = pylas.read(input_laz_path)
   las = pylas.convert(laz)
   a=b.replace('.laz','.las')
   output_las_path = '/content/output/' + a
   print(output_las_path)
   las.write(output_las_path)
   las = laspy.read(output_las_path)
   Lidar_data_points=pd.DataFrame(np.round(np.array(las.x),4),columns=['x']) 
   Lidar_data_points['y']=np.array(np.round(np.array(las.y),4))
   Lidar_data_points['z']=np.array(las.z)
   Lidar_data_points['X']=np.array(las.X)
   Lidar_data_points['Y']=np.array(las.Y)
   Lidar_data_points['classification']=np.array(las.classification)
   Lidar_data_points['gps_time']=las.gps_time
   os.remove(output_las_path) 
   dataset1 = Lidar_data_points[(Lidar_data_points.classification == 9)]
   dataset1['Time'] = dataset1['gps_time'].apply(gps_time_conversion)
   finaldata=dataset1.iloc[::20, :]
   filename= a.replace('.las','.csv')
   print(filename)
   client = storage.Client()
   bucket = client.get_bucket('slr-test-bucket')
   bucket.blob('Sanddiego2015_CSV/'+ filename).upload_from_string(finaldata.to_csv(), 'text/csv') 
   os.remove(input_laz_path)
